<font face="微软雅黑" size=5 color=#A52A2A >Naïve Bayes 分类器

In [1]:
import numpy as np
import pandas as pd

#待处理的数据
data = [['Sunny','Hot','High','Weak','No'],['Sunny','Hot','High','Strong','No'],\
        ['Overcast','Hot','High','Weak','Yes'],['Rain','Mild','High','Weak','Yes'],\
        ['Rain','Cool','Normal','Weak','Yes'],['Rain', 'Cool', 'Normal', 'Strong', 'No'],\
        ['Overcast','Cool','Normal','Strong','Yes'],['Sunny','Mild','High','Weak','No'],\
        ['Sunny','Cool','Normal','Weak','Yes'],['Rain','Mild','Normal','Weak','Yes'],\
        ['Sunny','Mild','Normal','Strong','Yes'],['Overcast','Mild','High','Strong','Yes'],\
        ['Overcast','Hot','Normal','Weak','Yes'],['Rain','Mild','High','Strong','No']]

Data = pd.DataFrame(data,columns=['Outlook','Temperature','Humidity','Wind','PlayTennis']) #属性名


<font face="显示数据" size=5 color=#A52A2A >

<font face="微软雅黑" size=5 color=#A52A2A >显示数据

In [2]:
Data.head()

,Outlook,Temperature,Humidity,Wind,PlayTennis
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes


In [3]:
cols = Data.shape[1]
X_data = Data.iloc[:,:cols-1]
Y_data = Data.iloc[:,cols-1:]
featureNames = X_data.columns

<font face="微软雅黑" size=5 color=#A52A2A >Naïve_Bayes类

In [4]:
class Naïve_Bayes:
    #训练朴素贝叶斯模型
    def Naive_Bayes_train(self,x_data, y_data):
        y = y_data.values
        x = x_data.values
        y_unique = np.unique(y) #获取类别标签值
        prior_prob = np.zeros(len(y_unique)) #存储先验概率

        #计算先验概率
        for i in range(len(y_unique)):
            prior_prob[i] = np.sum(y == y_unique[i]) / len(y)

        condition_prob = {} #似然关系的字典

        #计算似然关系
        for feat in featureNames:
            x_unique = list(set(x_data[feat]))
            x_condition_prob = np.zeros((len(y_unique), len(x_unique)))
            for j in range(len(y_unique)):
                for k in range(len(x_unique)):
                    x_condition_prob[j, k] = np.sum((x_data[feat] == x_unique[k]) & (y_data['PlayTennis'] == y_unique[j])) / np.sum(y == y_unique[j])
            x_condition_prob = pd.DataFrame(x_condition_prob, columns=x_unique, index=y_unique)
            condition_prob[feat] = x_condition_prob #将条件概率存入字典

        return prior_prob, condition_prob

    #预测未知样本
    def Prediction(self,testdata, prior_p, condition_prob):
        class_num = len(prior_p)
        featureNames = testdata.columns
        sample_num = testdata.shape[0]
        post_prob = np.zeros((sample_num, class_num)) #记录测试样例,在各类别下的后验概率

        for k in range(sample_num):
            prob_k = np.zeros((class_num,))
            for i in range(class_num):
                pri = prior_p[i]
                for feat in featureNames:
                    feat_val = testdata[feat][k]
                    cp = condition_prob[feat]
                    cp_val = cp[feat_val].iloc[i]
                    pri *= cp_val
                prob_k[i] = pri
            prob = prob_k / np.sum(prob_k, axis=0)
            post_prob[k, :] = prob
        return post_prob

<font face="微软雅黑" size=5 color=#A52A2A >训练模型

In [5]:
na = Naïve_Bayes #创建一个类的实例化
prior,condition = na.Naive_Bayes_train(na,X_data,Y_data) #训练模型

<font face="微软雅黑" size=5 color=#A52A2A >似然关系和先验概率

In [6]:
prior

array([0.35714286, 0.64285714])

In [7]:
condition['Outlook']

,Rain,Overcast,Sunny
No,0.400000,0.000000,0.600000
Yes,0.333333,0.444444,0.222222


In [8]:
condition['Temperature']

,Cool,Mild,Hot
No,0.200000,0.400000,0.400000
Yes,0.333333,0.444444,0.222222


In [9]:
condition['Humidity']

,High,Normal
No,0.800000,0.200000
Yes,0.333333,0.666667


In [10]:
condition['Wind']

,Strong,Weak
No,0.600000,0.400000
Yes,0.333333,0.666667


<font face="微软雅黑" size=5 color=#A52A2A >对未知样本进行预测

In [11]:
testdata = [['Sunny', 'Cool', 'High', 'Strong']] #测试样本
testdata = pd.DataFrame(testdata, columns=['Outlook', 'Temperature', 'Humidity', 'Wind'])

In [12]:
postPrior = na.Prediction(na,testdata, prior, condition) #对样本进行预测
postPrior

array([[0.79541735, 0.20458265]])

<font face="微软雅黑" size=5 color=#A52A2A >预测为No